In [1]:
import pandas as pd

In [28]:
import warnings
warnings.filterwarnings("ignore")

In [2]:
df = pd.read_csv('year_joined.csv')

Кроме того, нужно удостовериться, что данные означают именно то, что, по вашему мнению, должны означать. Допустим, что вы хотите удостовериться, что текущий статус определяется его годовым, а не другим, более краткосрочным статусом. Один из способов ответить на этот вопрос — проверить, есть ли у членов организации более одной записи

In [3]:
df.groupby('memberID').count().groupby('memberStats').count()

,yearJoined
memberStats,
1,1000


Здесь мы видим, что все 1000 членов имеют только один статус, поэтому для всех 
них показатель YearJoined, скорее всего, будет указывать год вступления в
организацию, а по нему будет определяться текущий статус или статус на момент
вступления в организацию. 

In [6]:
df1 = pd.read_csv('emails.csv')

есть ли в таблице недели, в которые участник не открыл ни одного электронного письма?

In [7]:
df1[df1.emailsOpened < 1]

,emailsOpened,member,week


просмотреть историю работы с электронной почтой одного пользователя

In [8]:
df1[df1.member==998]

,emailsOpened,member,week
25464,1.0,998.0,2017-12-04 00:00:00
25465,3.0,998.0,2017-12-11 00:00:00
25466,3.0,998.0,2017-12-18 00:00:00
25467,3.0,998.0,2018-01-01 00:00:00
25468,3.0,998.0,2018-01-08 00:00:00
25469,2.0,998.0,2018-01-15 00:00:00
25470,3.0,998.0,2018-01-22 00:00:00
25471,2.0,998.0,2018-01-29 00:00:00
25472,3.0,998.0,2018-02-05 00:00:00
25473,3.0,998.0,2018-02-12 00:00:00


количество еженедельных наблюдений между первым и последним событиями для рассматриваемого члена организации

In [9]:
import datetime as dt


In [10]:
a1 = dt.datetime.strptime(max(df1[df1.member == 998].week), "%Y-%m-%d %H:%M:%S")
a2 = dt.datetime.strptime(min(df1[df1.member == 998].week),"%Y-%m-%d %H:%M:%S")


In [11]:
(a1-a2).days/7

25.0

определить количество недель, для которых заданы учетные данные.

In [12]:
df1[df1.member == 998].shape

(24, 3)

заполнить все недостающие недели для всех участников сразу

In [13]:
complete_idx = pd.MultiIndex.from_product((set(df1.week), set(df1.member)))
complete_idx

MultiIndex([('2015-10-26 00:00:00',   1.0),
            ('2015-10-26 00:00:00',   3.0),
            ('2015-10-26 00:00:00',   5.0),
            ('2015-10-26 00:00:00',   6.0),
            ('2015-10-26 00:00:00',   9.0),
            ('2015-10-26 00:00:00',  10.0),
            ('2015-10-26 00:00:00',  14.0),
            ('2015-10-26 00:00:00',  16.0),
            ('2015-10-26 00:00:00',  20.0),
            ('2015-10-26 00:00:00',  21.0),
            ...
            ('2017-08-21 00:00:00', 973.0),
            ('2017-08-21 00:00:00', 977.0),
            ('2017-08-21 00:00:00', 982.0),
            ('2017-08-21 00:00:00', 984.0),
            ('2017-08-21 00:00:00', 987.0),
            ('2017-08-21 00:00:00', 991.0),
            ('2017-08-21 00:00:00', 992.0),
            ('2017-08-21 00:00:00', 993.0),
            ('2017-08-21 00:00:00', 995.0),
            ('2017-08-21 00:00:00', 998.0)],
           length=93247)

Используем полученный индекс для переиндексации исходной таблицы и заполнения пропущенных значений — в нашем случае нулем, 
предполагая, что если в строке ничего не записано, значит, записывать было нечего. Мы также сбрасываем индекс, чтобы заносить информацию
о членах организации и неделях в отдельные столбцы, а затем присваиваем имена этим столбцам.

In [14]:
all_email = df1.set_index([ 'week', 'member']).reindex(complete_idx, fill_value = 0).reset_index()
all_email.columns = ['week', 'member', 'emailsOpened' ]
all_email

,week,member,emailsOpened
0,2015-10-26 00:00:00,1.0,3.0
1,2015-10-26 00:00:00,3.0,0.0
2,2015-10-26 00:00:00,5.0,0.0
3,2015-10-26 00:00:00,6.0,0.0
4,2015-10-26 00:00:00,9.0,0.0
...,...,...,...
93242,2017-08-21 00:00:00,991.0,0.0
93243,2017-08-21 00:00:00,992.0,0.0
93244,2017-08-21 00:00:00,993.0,0.0
93245,2017-08-21 00:00:00,995.0,0.0


изучим данные для члена организации с индексом 998

In [15]:
all_email[all_email.member == 998].sort_values('week')

,week,member,emailsOpened
5389,2015-02-09 00:00:00,998.0,0.0
29644,2015-02-16 00:00:00,998.0,0.0
24254,2015-02-23 00:00:00,998.0,0.0
67374,2015-03-02 00:00:00,998.0,0.0
18864,2015-03-09 00:00:00,998.0,0.0
...,...,...,...
85161,2018-04-30 00:00:00,998.0,3.0
30722,2018-05-07 00:00:00,998.0,3.0
38268,2018-05-14 00:00:00,998.0,3.0
91090,2018-05-21 00:00:00,998.0,3.0


Для каждого члена организации определим срезы start_date и end_date, группируя DataFrame электронной
почты для каждого члена организации и выбирая максимальное и минимальное значения недели

In [16]:
cutoff_dates = df1.groupby('member').week.agg(['min', 'max' ]).reset_index()
cutoff_dates = cutoff_dates.reset_index()
cutoff_dates

,index,member,min,max
0,0,1.0,2015-06-29 00:00:00,2018-05-28 00:00:00
1,1,3.0,2018-03-05 00:00:00,2018-04-23 00:00:00
2,2,5.0,2017-06-05 00:00:00,2018-05-28 00:00:00
3,3,6.0,2016-12-05 00:00:00,2018-05-28 00:00:00
4,4,9.0,2016-07-18 00:00:00,2018-05-28 00:00:00
...,...,...,...,...
534,534,991.0,2016-10-24 00:00:00,2016-10-24 00:00:00
535,535,992.0,2015-02-09 00:00:00,2015-07-06 00:00:00
536,536,993.0,2017-09-11 00:00:00,2018-05-28 00:00:00
537,537,995.0,2016-09-05 00:00:00,2018-05-28 00:00:00


Удалим из DataFrame строки, которые не вносят заметного вклада в хронологию
событий, а именно — все нулевые строки перед первыми ненулевыми строками
каждого члена организации

In [29]:
for _, row in cutoff_dates.iterrows():
    member = row ['member']
    start_date = row ['min']
    end_date = row ['max']
    all_email.drop(all_email[all_email.member == member][all_email.week < start_date].index, inplace=True)
    all_email.drop(all_email[all_email.member == member][all_email.week > end_date].index, inplace=True)

In [25]:
all_email

,week,member,emailsOpened
0,2015-10-26 00:00:00,1.0,3.0
13,2015-10-26 00:00:00,31.0,3.0
14,2015-10-26 00:00:00,33.0,2.0
15,2015-10-26 00:00:00,34.0,3.0
16,2015-10-26 00:00:00,38.0,1.0
...,...,...,...
93236,2017-08-21 00:00:00,970.0,3.0
93237,2017-08-21 00:00:00,973.0,2.0
93240,2017-08-21 00:00:00,984.0,0.0
93241,2017-08-21 00:00:00,987.0,3.0


файл donations

In [18]:
donations = pd.read_csv('donations.csv')

В этом коде мы сначала преобразуем строковый символ в соответствующий класс данных с временными метками, чтобы в дальнейшем применить функцию индексации по дате пакета Pandas.

Наконец, в получаемых для каждого члена организации фреймах данных мы группируем и суммируем взносы за недельные периоды времени, отбрасываем недели, в которых не оплачивались взносы, а затем складываем их вместе

In [30]:
donations.timestamp = pd.to_datetime(donations.timestamp)
donations.set_index('timestamp', inplace = True)
agg_don = donations.groupby('member').apply(lambda df: df.amount.resample ("W-MON").sum().dropna())


In [31]:
donations

,amount,member
timestamp,,
2017-11-12 11:13:44,25.0,0.0
2015-08-25 19:01:45,50.0,0.0
2015-03-26 12:03:47,25.0,0.0
2016-07-06 12:24:55,50.0,0.0
2016-05-11 18:13:04,50.0,1.0
...,...,...
2016-09-02 11:20:00,25.0,992.0
2017-11-02 12:17:06,50.0,993.0
2016-09-13 21:09:47,1000.0,995.0


In [44]:
agg_donation=pd.DataFrame(agg_don)

In [56]:
agg_donation

amount
member timestamp         
0.0    2015-03-30    25.0
       2015-04-06     0.0
       2015-04-13     0.0
       2015-04-20     0.0
       2015-04-27     0.0
...                   ...
995.0  2017-09-11     0.0
       2017-09-18     0.0
       2017-09-25     0.0
       2017-10-02  1000.0
998.0  2018-01-08    50.0

[32352 rows x 1 columns]

Получив выборки данных о взносах и работе с электронной почтой, имеющие одинаковую периодичность, объединим их.

In [62]:
merged_df = pd.DataFrame()


In [63]:
for member, member_email in all_email.groupby('member') :
    member_donations = donations[donations.member == member]
    #member_donations.set_index('timestamp', inplace = True)
    member_email.set_index ('week', inplace = True)
    member_email = all_email[all_email.member == member]
    member_email.sort_values('week').set_index('week')
    df = pd.merge(member_email, member_donations, how = 'left', left_index = True, right_index = True)
    df.fillna(0)
    df['member'] = df.member_x
    merged_df = merged_df.append(df.reset_index()[['member', 'week', 'emailsOpened', 'amount']] )


In [66]:
df = merged_df[merged_df.member == 998]
df['target'] = df.amount.shift(1)
df = df.fillna(0)
df.set_index('member', inplace = True)
df.sort_values('week')


,week,emailsOpened,amount,target
member,,,,
998.0,2017-12-04 00:00:00,1.0,0.0,0.0
998.0,2017-12-11 00:00:00,3.0,0.0,0.0
998.0,2017-12-18 00:00:00,3.0,0.0,0.0
998.0,2017-12-25 00:00:00,0.0,0.0,0.0
998.0,2018-01-01 00:00:00,3.0,0.0,0.0
998.0,2018-01-08 00:00:00,3.0,0.0,0.0
998.0,2018-01-15 00:00:00,2.0,0.0,0.0
998.0,2018-01-22 00:00:00,3.0,0.0,0.0
998.0,2018-01-29 00:00:00,2.0,0.0,0.0


Итак, после учета пропущенных строк данные члена организации с идентификатором 998 стали насчитывать 26 строк. Набор данных стал полным и очистился от ненужной информации.

**Часовые пояса**

In [2]:
import datetime
datetime.datetime.utcnow()
datetime.datetime(2018, 5, 31, 14, 49, 43, 187680)
datetime.datetime.now()
datetime.datetime(2018, 5, 31, 10, 49, 59, 984947)
# Компьютер не возвращает время в формате UTC,
# хотя часовой пояс не указан
datetime.datetime.now (datetime. timezone.utc)
datetime.datetime(2018, 5, 31, 14, 51, 35, 601355,
tzinfo=datetime.timezone.utc)


datetime.datetime(2018, 5, 31, 14, 51, 35, 601355, tzinfo=datetime.timezone.utc)

Чтобы получить правильный результат, достаточно перейти к формату с указанием часового пояса, но это нестандартное поведение. Для управления часовыми поясами в языке Python создается объект часового пояса, например western, представляющий тихоокеанский часовой пояс США.


In [4]:
import pytz
western = pytz.timezone('US/Pacific')
western.zone


'US/Pacific'

In [10]:
loc_dt = western.localize(datetime.datetime(2018, 5, 15, 12, 34, 0))
london_dt = pytz.timezone('Europe/London')
london_dt = loc_dt.astimezone(london_tz)
london_dt


datetime.datetime(2018, 5, 15, 20, 34, tzinfo=<DstTzInfo 'Europe/London' BST+1:00:00 DST>)

Однако не забывайте, что передача часового пояса непосредственно в конструктор datetime() часто приводит к совершенно неожиданному результату.


In [12]:
# Как правило, данные времени хранятся в формате UTC
# и преобразовываются в другие форматы только при выводе
# для изучения людьми. Кроме того, к часовым поясам часто
# применяются арифметические операции
event1 = datetime.datetime(2018, 5, 12, 12, 15, 0, tzinfo = london_tz)
event2 = datetime.datetime (2018, 5, 13, 9, 15, 0,tzinfo = western)
event2 - event1



datetime.timedelta(days=1, seconds=17520)

In [14]:
## Разница между моментами времени определяется неправильно,
## поскольку часовые пояса помечены не так, как нужно
eventl = london_tz.localize(datetime.datetime(2018, 5, 12, 12, 15, 0))
event2 = western.localize(datetime.datetime(2018, 5, 13, 9, 15, 0))
event2 - event1

datetime.timedelta(days=1, seconds=14340)

In [15]:
event1 = london_tz.localize((datetime.datetime(2018, 5, 12, 12, 15, 0))).astimezone(datetime.timezone.utc)
event2 = western.localize(datetime.datetime(2018, 5, 13, 9, 15, 0)).astimezone(datetime.timezone.utc)
event2 - event1

datetime.timedelta(days=1, seconds=18000)

Библиотека pytz включает списки наиболее распространенных часовых поясов и часовых поясов разных стран мира.

In [18]:
## Ознакомьтесь основными часовыми поясами pytz.common_timezones
pytz.common_timezones

## или часовыми поясами стран мира


pytz.country_timezones('eu')


['Europe/Kaliningrad',
 'Europe/Moscow',
 'Europe/Kirov',
 'Europe/Volgograd',
 'Europe/Astrakhan',
 'Europe/Saratov',
 'Europe/Ulyanovsk',
 'Europe/Samara',
 'Asia/Yekaterinburg',
 'Asia/Omsk',
 'Asia/Novosibirsk',
 'Asia/Barnaul',
 'Asia/Tomsk',
 'Asia/Novokuznetsk',
 'Asia/Krasnoyarsk',
 'Asia/Irkutsk',
 'Asia/Chita',
 'Asia/Yakutsk',
 'Asia/Khandyga',
 'Asia/Vladivostok',
 'Asia/Ust-Nera',
 'Asia/Magadan',
 'Asia/Sakhalin',
 'Asia/Srednekolymsk',
 'Asia/Kamchatka',
 'Asia/Anadyr']

Особенно проблематичным является вопрос учета летнего времени. Некоторые используемые человечеством форматы времени учитываются компьютером дважды (возврат к “зимнему” времени осенью), тогда как другие вообще не учитываются (перевод времени вперед весной).

In [20]:
## Часовые пояса
ambig_time = western.localize(datetime.datetime(2002, 10, 27, 1, 30, 00)).astimezone(datetime.timezone.utc)
ambig_time_earlier = ambig_time - datetime.timedelta(hours=1)
ambig_time_later = ambig_time + datetime.timedelta(hours=1)
ambig_time_earlier.astimezone(western)



datetime.datetime(2002, 10, 27, 1, 30, tzinfo=<DstTzInfo 'US/Pacific' PDT-1 day, 17:00:00 DST>)

In [21]:
ambig_time.astimezone (western)

datetime.datetime(2002, 10, 27, 1, 30, tzinfo=<DstTzInfo 'US/Pacific' PST-1 day, 16:00:00 STD>)

In [22]:
ambig_time_later.astimezone (western)

datetime.datetime(2002, 10, 27, 2, 30, tzinfo=<DstTzInfo 'US/Pacific' PST-1 day, 16:00:00 STD>)

In [26]:
# Последние две временные метки идентичны - это недопустимо!
## В данном случае необходимо использовать параметр is_dst, чтобы
## обозначить переход на летнее время
ambig_time = western.localize(datetime.datetime(2002, 10, 27, 1, 30, 00), is_dst = True).astimezone(datetime.timezone.utc)
ambig_time_earlier = ambig_time - datetime.timedelta (hours=1)
ambig_time_later = ambig_time + datetime.timedelta(hours=1)
ambig_time_earlier.astimezone(western)



datetime.datetime(2002, 10, 27, 0, 30, tzinfo=<DstTzInfo 'US/Pacific' PDT-1 day, 17:00:00 DST>)

In [27]:
ambig_time.astimezone (western)


datetime.datetime(2002, 10, 27, 1, 30, tzinfo=<DstTzInfo 'US/Pacific' PDT-1 day, 17:00:00 DST>)

In [28]:

ambig_time_later.astimezone(western)

datetime.datetime(2002, 10, 27, 1, 30, tzinfo=<DstTzInfo 'US/Pacific' PST-1 day, 16:00:00 STD>)

Трудности учета часового пояса могут не касаться решаемых вами задач, поэтому полезность приведенных выше сведений будет зависеть от характера обрабатываемых данных. 